In [12]:
import pandas as pd
import re
from tqdm import tqdm
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

In [3]:
PRO_ISRAELI =[ "cabinet", "government", "homeland", "humanitarian aid", "idf", "iron dome",
    "israel", "israeli", "jerusalem", "jewish", "knesset", "mossad", "netanyahu",
    "occupation", "occupied territories", "parliament", "settlers", "tel aviv",
    "west bank", "zionism", "zionist entity", "zionist regime", "zionist state",
    "security", "defense forces", "coexistence", "peace", "innovation", "startup nation",
    "sabra", "heritage", "aliyah", "yom hashoah", "yom hazikaron", "yom haatzmaut",
    "negev", "ashkelon", "ashdod", "haifa", "eilat", "ben gurion", "sderot",
    "technological advancement", "diplomacy", "jewish agency", "menorah",
    "western wall", "temple mount", "david's star", "dead sea", "red sea",
    "kinneret", "masada", "hebrew", "torah", "synagogue", "economic growth",
    "resilience", "chutzpah", "maccabees", "diaspora", "el al", "hatikvah"
]

PRO_PALESTINIAN = [
    "abbas", "colonizers", "displaced", "freedom fighters", "gaza", "gazans",
    "hamas", "hassan nasrallah", "hezbollah", "houthis", "humanitarian crisis",
    "intifada", "iran", "muhammad sinuar", "naim qassem", "nakba", "nukhba",
    "oppressed", "organization", "palestine", "palestinians", "plo", "refugees",
    "resistance", "resisters", "sinuar", "terrorists", "tyrants", "checkpoint",
    "apartheid", "liberation", "sovereignty", "right of return", "unrwa",
    "blockade", "land day", "sheikh jarrah", "solidarity", "intifada stones",
    "ceasefire", "al quds", "naksa", "siege", "exile", "ethnic cleansing",
    "political prisoners", "self-determination", "separation wall", "war crimes",
    "genocide", "protest", "justice", "humanitarian assistance", "freedom march",
    "jenin", "rafah", "khan younis", "beit hanoun", "hebron", "bethlehem",
    "nablus", "ramallah", "al-awda", "jerusalem", "al-naksa", "martyr",
    "international law", "un resolution", "settler violence", "deir yassin",
    "sabra and shatila", "olive trees", "revolution", "human rights"
]

In [4]:
# Load the Excel file
def process_text():
  file_path = "posts_first_targil.xlsx"
  df = pd.read_excel(file_path, sheet_name=None)

  # Initialize the new DataFrame
  df_sentence = pd.DataFrame(columns=['sheet_name', 'id', 'sentence'])

  # Process each sheet in the Excel file
  for sheet_name, data in df.items():
      for index, row in data.iterrows():
          # Combine text based on the sheet
          if sheet_name == 'A-J':
              combined_text = " ".join(str(row[col]) for col in ['title', 'sub_title', 'Body Text'] if col in row and pd.notna(row[col]))
          else:
              combined_text = " ".join(str(row[col]) for col in ['title', 'Body Text'] if col in row and pd.notna(row[col]))

          # Split the combined text into sentences
          sentences = [sentence.strip() for sentence in re.split(r'[.!?]', combined_text) if sentence.strip()]

          # Append sentences to the new DataFrame
          for sentence in sentences:
              df_sentence = pd.concat([df_sentence, pd.DataFrame({ 'sheet_name': [sheet_name], 'id': [index], 'sentence': [sentence]})], ignore_index=True)
  df_sentence.head()
  return df_sentence

data = process_text()
print(len(data))


30125


In [5]:
def extract_sentences(df, pro_israeli_words, pro_palestinian_words):
    extracted = []

    for idx, row in df.iterrows():
        doc_id = row['id']

        sentence = row['sentence'].strip().lower()
        sheet_name = row['sheet_name']
        # Check for pro-Israeli and pro-Palestinian keywords
        is_pro_israeli = any(word in sentence for word in pro_israeli_words)
        is_pro_palestinian = any(word in sentence for word in pro_palestinian_words)

        # Determine the type based on keywords
        if is_pro_israeli and not is_pro_palestinian:
            extracted.append((doc_id,sheet_name, sentence, 'pro-israeli'))
        elif is_pro_palestinian and not is_pro_israeli:
            extracted.append((doc_id,sheet_name, sentence, 'pro-palestinian'))

    return pd.DataFrame(extracted, columns=['id','sheet_name', 'sentence', 'lebal'])

data = extract_sentences(data,PRO_ISRAELI,PRO_PALESTINIAN)
print(len(data))

9564


In [15]:


def sentiment_models():
    model_paths = {
        'model1': "cardiffnlp/twitter-roberta-base-sentiment-latest",
        'model2': "nlptown/bert-base-multilingual-uncased-sentiment",
        'model3': "cardiffnlp/twitter-xlm-roberta-base-sentiment",
        'model4': "siebert/sentiment-roberta-large-english",
        'model5': "lxyuan/distilbert-base-multilingual-cased-sentiments-student",
        'model6': "finiteautomata/bertweet-base-sentiment-analysis",
        'model7': "j-hartmann/sentiment-roberta-large-english-3-classes"
    }

    sent_models = {}

    for name, path in model_paths.items():
        try:
            if name == 'model3':
                # Explicitly handle model3 with slow tokenizer if necessary
                tokenizer = AutoTokenizer.from_pretrained(path, use_fast=False)  # Use slow tokenizer for SentencePiece
                model = AutoModelForSequenceClassification.from_pretrained(path)
                sent_models[name] = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
            else:
                # Default pipeline loading for other models
                sent_models[name] = pipeline("sentiment-analysis", model=path, tokenizer=path)
            print(f"{name}: Loaded successfully")
        except Exception as e:
            print(f"{name}: Failed to load - {e}")
            sent_models[name] = None

    return sent_models

# Initialize and test models
models = sentiment_models()


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model1: Loaded successfully
model2: Loaded successfully
model3: Failed to load - Converting from Tiktoken failed, if a converter for SentencePiece is available, provide a model path with a SentencePiece tokenizer.model file.Currently available slow->fast convertors: ['AlbertTokenizer', 'BartTokenizer', 'BarthezTokenizer', 'BertTokenizer', 'BigBirdTokenizer', 'BlenderbotTokenizer', 'CamembertTokenizer', 'CLIPTokenizer', 'CodeGenTokenizer', 'ConvBertTokenizer', 'DebertaTokenizer', 'DebertaV2Tokenizer', 'DistilBertTokenizer', 'DPRReaderTokenizer', 'DPRQuestionEncoderTokenizer', 'DPRContextEncoderTokenizer', 'ElectraTokenizer', 'FNetTokenizer', 'FunnelTokenizer', 'GPT2Tokenizer', 'HerbertTokenizer', 'LayoutLMTokenizer', 'LayoutLMv2Tokenizer', 'LayoutLMv3Tokenizer', 'LayoutXLMTokenizer', 'LongformerTokenizer', 'LEDTokenizer', 'LxmertTokenizer', 'MarkupLMTokenizer', 'MBartTokenizer', 'MBart50Tokenizer', 'MPNetTokenizer', 'MobileBertTokenizer', 'MvpTokenizer', 'NllbTokenizer', 'OpenAIGPTToken

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


model6: Loaded successfully


Some weights of the model checkpoint at j-hartmann/sentiment-roberta-large-english-3-classes were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model7: Loaded successfully


In [ ]:
def map_sentiment_to_context(sentiment_label, score, sentence_context):
    """Map sentiment label to pro-Israeli/pro-Palestinian context."""

    # Normalize sentiment label
    sentiment = sentiment_label.lower()

    # Neutral sentiment handling
    if 'neutral' in sentiment:
        return 'NEUTRAL'

    # Map sentiment based on sentence context
    if sentence_context == 'pro-israeli':
        return 'POS' if sentiment in ['positive', 'pos'] else 'NEG'
    elif sentence_context == 'pro-palestinian':
        return 'POS' if sentiment in ['positive', 'pos'] else 'NEG'

    # Default to NEUTRAL if no match
    return 'NEUTRAL'


def evaluate_sentence_with_models(sentence, context_type, models, tokenizer_name="bert-base-uncased"):
    """
    Evaluate a sentence using multiple models, splitting it into chunks if necessary.

    Args:
        sentence (str): The input sentence to evaluate.
        context_type (str): The context type (e.g., 'pro-israeli', 'pro-palestinian').
        models (dict): A dictionary of model names and their instances.
        tokenizer_name (str): The tokenizer to use for token length calculation.

    Returns:
        dict: A dictionary of model results including scores and labels.
    """
    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    # Tokenize the sentence and check its length
    tokenized_sentence = tokenizer.encode(sentence, add_special_tokens=True)
    if len(tokenized_sentence) <= 512:
        # Sentence fits within token limits
        return _evaluate_single_sentence(sentence, context_type, models)
    else:
        # Split the sentence into smaller chunks
        chunks = _split_into_chunks(sentence, tokenizer, max_length=512)
        aggregated_results = {}

        for chunk in chunks:
            chunk_results = _evaluate_single_sentence(chunk, context_type, models)
            for key, value in chunk_results.items():
                if key in aggregated_results:
                    aggregated_results[key].append(value)
                else:
                    aggregated_results[key] = [value]

        # Aggregate results across chunks
        return _aggregate_chunk_results(aggregated_results)


def _evaluate_single_sentence(sentence, context_type, models):
    """
    Evaluate a single sentence using multiple models.
    """
    evaluation_results = {}

    for model_name, model_instance in models.items():
        try:
            prediction = model_instance(sentence)[0]
            evaluation_results[f"{model_name}_score"] = prediction['score']
            evaluation_results[f"{model_name}_label"] = map_sentiment_to_context(
                prediction['label'], prediction['score'], context_type
            )
        except Exception as e:
            print(f"Error with model '{model_name}' for sentence: '{sentence[:50]}...' Error: {e}")
            evaluation_results[f"{model_name}_score"] = None
            evaluation_results[f"{model_name}_label"] = None

    return evaluation_results


def _split_into_chunks(sentence, tokenizer, max_length):
    """
    Split a sentence into chunks that fit within the token limit.

    Args:
        sentence (str): The sentence to split.
        tokenizer: The tokenizer instance.
        max_length (int): The maximum number of tokens per chunk.

    Returns:
        list: A list of sentence chunks.
    """
    words = sentence.split()
    chunks = []
    current_chunk = []

    for word in words:
        # Add word to the current chunk and check its token length
        current_chunk.append(word)
        if len(tokenizer.encode(" ".join(current_chunk), add_special_tokens=True)) > max_length:
            # Remove the last word and finalize the current chunk
            current_chunk.pop()
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]

    # Add the final chunk
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks


def _aggregate_chunk_results(results):
    """
    Aggregate results across multiple chunks by averaging scores and determining majority labels.
    """
    aggregated = {}

    for key, values in results.items():
        if '_score' in key:
            # Average scores
            aggregated[key] = sum(values) / len(values)
        elif '_label' in key:
            # Majority label
            aggregated[key] = max(set(values), key=values.count)

    return aggregated

def determine_majority_sentiment(row):
    """Determine the majority sentiment label across multiple models."""
    sentiment_labels = [value for key, value in row.items() if '_label' in key and value is not None]

    if not sentiment_labels:
        return 'UNKNOWN'

    # Determine majority label
    majority_label = max(set(sentiment_labels), key=sentiment_labels.count)
    return majority_label

In [ ]:
# Process each sentence
results = []
for idx, row in tqdm(data.iterrows(), total=len(data)):
    result = {
        'newspaper': row['sheet_name'].split('_')[0],
        'article_id': row['id'],
        'sentence': row['sentence'],
        'type': row['lebal']
    }

    # Add model predictions
    result.update(evaluate_sentence_with_models(row['sentence'], row['lebal'], models))

    # Add to results
    results.append(result)

# Create final DataFrame
output_df = pd.DataFrame(results)

# Add majority decision
output_df['majority_decision'] = output_df.apply(determine_majority_sentiment, axis=1)

# Calculate average score for majority decision
score_columns = [col for col in output_df.columns if '_score' in col]
output_df['avg_majority_score'] = output_df[score_columns].mean(axis=1)
# Save to Excel
output_df.to_excel('sentiment_results.xlsx', index=False)
